In [1]:
import geocoder
import pandas as pd
import codecs

In [2]:
with codecs.open("./catch_data.csv", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_table(file, delimiter=",")
    print(df)

           年    漁港 都道府県      地域       項目 上場水揚量幅  上場水揚量    価格  合計or個別
0       2016    鞆浦   徳島  太平洋南区計  あかいか（生）      0      -    -        0
1       2017    鞆浦   徳島  太平洋南区計  あかいか（生）      0      -     0       0
2       2018    鞆浦   徳島  太平洋南区計  あかいか（生）      0      -     0       0
3       2019    鞆浦   徳島  太平洋南区計  あかいか（生）      0      -     0       0
4       2016    亘理   宮城  太平洋北区計  あかいか（生）      0      -    -        0
...      ...   ...  ...     ...      ...    ...     ...  ...     ...
270148  2011  おいらせ   青森  太平洋北区計     冷凍品計      0      -    -        1
270149  2012  おいらせ   青森  太平洋北区計     冷凍品計      0      -    -        1
270150  2013  おいらせ   青森  太平洋北区計     冷凍品計      0      -    -        1
270151  2014  おいらせ   青森  太平洋北区計     冷凍品計      0      -    -        1
270152  2015  おいらせ   青森  太平洋北区計     冷凍品計      0      -    -        1

[270153 rows x 9 columns]


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
from typing import NewType, Optional, Tuple, List

Lat = NewType('Lat', str)
Lng = NewType('Lng', str)
LatLng = Tuple[Lat, Lng]

def getLatLng(place_name: str) -> LatLng or None:
    try:
        ret = geocoder.osm(place_name, timeout=5.0)
        latlng = (ret.latlng[0], ret.latlng[1])
        return latlng
    except:
        return None

In [ ]:
df["LatLng"] = df["漁港"].map(getLatLng)

In [ ]:
df.to_csv("./catch_with_latlng.csv")

In [ ]:
!pip install cartopy

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import itertools

In [ ]:
df = pd.read_csv('./catch_with_latlng.csv')

In [ ]:
import typing
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from operator import add

max_marker_size = 20

def toTuple(string: str) -> typing.Tuple[float, float]:
    return tuple(map(float, string.strip("(").strip(")").split(', '))) 

def drawCircle(lat: float, lng: float, value: float) -> None:
    ax.plot(lng, lat, 'o', transform=ccrs.PlateCarree(),markersize=value, color = (1, 0, 0, 0.8))
    
def removeWhiteSpace(string: str) -> str:
    return string.replace(" ", "").strip("　")

def checkError(r, year):
    error = []
    isError = False
    
    if r["LatLng"] == "":
        error.append("No LatLng Data")
        isError = True
    try:
        latlng = toTuple(r["LatLng"])
    except:
        error.append("LatLng cannot convert to Tuple Type")
        isError = True
    if r[" 上場水揚量 "] == "- " or r[" 上場水揚量 "] == "-":
        error.append("No Catch Data")
        isError = True
    if r["年"] != year:
        error.append("This is not target year")
        isError = True
    return (error, isError)

def initializeMatplotArea(fig):
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=180))
    ax.coastlines(resolution="10m")
    ax.set_extent([150, 125, 50, 25], ccrs.PlateCarree())
    return ax

def drawAll(fig, draw_list, max_catch_size, year):
    ax = initializeMatplotArea(fig)
    
    ax.text(0.1, 0.9, year, ha='center', va='center', transform=ax.transAxes, fontsize=12)
    
    for draw_content in draw_list:
        latlng = draw_content["latlng"]
        catch_size = draw_content["catch_size"]
        ax.plot(latlng[1], latlng[0], 'o', transform=ccrs.PlateCarree(),markersize=(catch_size / max_catch_size) * max_marker_size , color = (1, 0, 0, 0.3))

In [ ]:
def gallery(array, ncols=3):
    nindex, height, width, intensity = array.shape
    nrows = nindex//ncols
    result = (array.reshape(nrows, ncols, height, width, intensity)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols, intensity))
    return result

def create_gallery(contentList):
    images = []
    files = sorted(glob.glob(f'./output/{contentList[0]}/*.png'))  
    yearList = list(map(lambda files: files.split("/")[-1].split(".")[0], files))
    for year in yearList:
        images = []
        for content in contentList:
            images.append(np.asarray(Image.open(f'./output/{content}/{year}.png').convert('RGB')))
        array = np.array(images)
        result = gallery(array, 3)
        im = Image.fromarray(result)
        im.save(f"./gallery/{year}.jpg")

    files = sorted(glob.glob(f'./gallery/*.jpg'))  

    images = list(map(lambda file : Image.open(file) , files))
    images[0].save(f'./gallery/output.gif' , save_all = True , append_images = images[1:] , duration = 1000 , loop = 0)

In [ ]:
create_gallery(["かつお（生）", "かつお（冷）" , "まぐろ（生）", "まぐろ（冷）", "ほっけ（生）", "さんま", "あまだい", "かき（むき身）", "かき（殻付）", "まいわし", "まあじ", "べにずわいがに"])